In [ ]:
import sys

# For managing relative imports from notebook
if '..' not in sys.path: sys.path.append('..')

import config.config as dfc
import deepfake.dfutillib as df

# Dataset Downloader

This Notebook is just for downloading the full dataset off of the kaggle website. You can do this by hand from your browser  
locally, I really wrote it for AWS to pull directly onto an EC2 or Sagemaker instance (but works for local machine too). 

This uses **wget** and requires a **cookies.txt** file (stoopid kaggle CLI wouldn't work for the real deal dataset of this  
competition, and I couldn't get requests to work either because there's an authentication step, it's not fully public URIs).

To get the cookies.txt file: login to the kaggle site (you'll probably need to join the competition too), then use this Chrome  
plugin to save the cookies.txt file: https://chrome.google.com/webstore/detail/cookiestxt/njabckikapfpffapmjgojcnbfjonfjfg?hl=en

**TODO:** will probably need to write code in here to push the dataset to S3 after download, unless just parking it on the  
EC2/Sagemaker disk ends up being cost effective enough or necessary for efficient enough training.

In [ ]:
# NOTE: notebook requires appropriate setup of deepfake.config 
# Be prepared for 500GB to get downloaded into your DATA_LAKE
# (and you'll prob need another 500GB there to preprocess 
# it all too, but that doesn't happen from this notebook).

data_lake, cookies_txt = dfc.DATA_LAKE, dfc.COOKIES_TXT
assert df.file_exists(cookies_txt), f"Cookies file: {cookies_txt} does not exist."

In [ ]:
# Verify your setup with this little cell. Just pulls a single sample MP4
# file into your DATA_LAKE, unzips it and then removes the archive file.

zipname = 'aassnaulhq.mp4.zip'
url = ('https://www.kaggle.com/c/deepfake-detection-challenge/download/'
       'Z1QcG4KHiVQaMucH9dng%2Fversions%2FvI13Sc4R0yAI121wogYv%2F'
       'directories%2Ftest_videos%2Ffiles%2Faassnaulhq.mp4')

!wget --load-cookies '{cookies_txt}' -O '{data_lake}/{zipname}' '{url}'
!unzip -q '{data_lake}/{zipname}' -d '{data_lake}' 
!rm '{data_lake}/{zipname}'


In [ ]:
# Download and extract (all 50) deepfake video file partitions.
# Note: kaggle urls and filenames are effectively hardcoded.

# Can set istart and istop (same indexing as range fcn) to pull a
# subsets of the partitions if something goes wrong in the middle.
def download_kaggle_data(istart=50, istop=None):
#{
    if istop is None: 
        istart, istop = 0, istart
        
    for i in range(istart, istop):
        zipname = f'dfdc_train_part_{i:02d}.zip'
        url = f'https://www.kaggle.com/c/16880/datadownload/{zipname}'
        !wget --load-cookies '{cookies_txt}' -O '{data_lake}/{zipname}' '{url}'
        !unzip -q '{data_lake}/{zipname}' -d '{data_lake}'
        !rm '{data_lake}/{zipname}'
#}

# NOTE: uncomment this line to download all partitions
# download_kaggle_data()

# These two partitions, 18 and 35, are slightly incomplete relative to 
# metadata files, but re-downloading then did not correct the issue.
# download_kaggle_data(18, 19)
# download_kaggle_data(35, 36)